In [52]:
%load_ext autoreload 
%autoreload 2
from lens_simulation import SimulationRunner


sim_runner = SimulationRunner.SimulationRunner("config.yaml")
sim_runner.initialise_simulation()
sim_runner.setup_simulation()
# sim_runner.run_simulations()

# Test github

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
--------------------------------------------------

Simulation Run: subtly-tight-zebra (dfef4f0d-42a3-4fc5-a911-be611f9f78c4)
Data: \\ad.monash.edu\home\User028\pcle0002\Documents\repos\lens_simulation\src\lens_simulation\log\subtly-tight-zebra
--------------------------------------------------

6 Lens Configurations. 3 Stage Configurations
Generating 18 Simulation Configurations


  0%|          | 0/6 [00:00<?, ?it/s]

{'lenses': [{'exponent': 0.0,
             'height': 7e-05,
             'medium': 'medium_1',
             'name': 'lens_1'},
            {'exponent': 2.0,
             'height': 7.5e-05,
             'medium': 'medium_1',
             'name': 'lens_2'}],
 'log_dir': '\\\\ad.monash.edu\\home\\User028\\pcle0002\\Documents\\repos\\lens_simulation\\src\\lens_simulation\\log\\subtly-tight-zebra',
 'mediums': [{'name': 'medium_1', 'refractive_index': 2.348},
             {'name': 'medium_2', 'refractive_index': 1.5}],
 'options': {'save': False, 'save_plot': True, 'verbose': False},
 'run_id': UUID('dfef4f0d-42a3-4fc5-a911-be611f9f78c4'),
 'run_petname': 'subtly-tight-zebra',
 'sim_parameters': {'A': 10000,
                    'pixel_size': 1e-06,
                    'sim_wavelength': 4.88e-07,
                    'sim_width': 0.0045},
 'stages': [{'finish_distance': 0.01,
             'lens': 'lens_1',
             'n_slices': 100,
             'options': {'focal_distance_multiple': 2.0,


## Simulation Class
Test example simulation



In [2]:
%load_ext autoreload
%autoreload 2
from lens_simulation import Simulation
import uuid
import os
import yaml

with open("config.yaml", "r") as f:
    conf = yaml.full_load(f)

run_id = uuid.uuid4()  # run_id is for when running a batch of sims, each sim has unique id
data_path = os.path.join(os.getcwd() , "log",  str(run_id))
config = {"run_id": run_id, "parameters": None, "log_dir": data_path, 
            "sim_parameters": conf["sim_parameters"], "options": conf["options"],
            "mediums": conf["mediums"], "lenses": conf["lenses"],
            "stages": conf["stages"]}

sim = Simulation.Simulation(config)
# sim.run_simulation()

# sim.propagate_wavefront()

TypeError: unsupported operand type(s) for /: 'list' and 'float'

In [ ]:
%load_ext autoreload
%autoreload 2

from lens_simulation.run_simulation import run_single_simulation


run_single_simulation("config.yaml")

In [14]:
from lens_simulation.SimulationUtils import SimulationStage, SimulationOptions, SimulationParameters
from lens_simulation.Lens import Lens, Medium
import numpy as np
from lens_simulation.Simulation import gen_sq_freq_arr_2d, generate_squared_frequency_array, pad_simulation, calculate_equivalent_focal_distance

import matplotlib.pyplot as plt
from lens_simulation import utils

from scipy import fftpack
from tqdm import tqdm

def propagate_wavefront(sim_stage, passed_wavefront=None):

    lens = sim_stage.lens
    output_medium = sim_stage.output
    n_slices = sim_stage.n_slices
    start_distance = sim_stage.start_distance
    finish_distance = sim_stage.finish_distance

    DEBUG = options.debug

    # print("LENS NDIM: ", lens.profile.ndim)
    # TODO: move this somewhere better
    # create 2D lens profile
    # lens.extrude_profile(1e-6)
    lens.revolve_profile()        

    # padding (width of lens on each side)
    if lens.profile.ndim == 1:
        # print("1D SIM")
        sim_profile = lens.profile
        pad_px = 0#lens.profile.shape[-1]
        sim_profile = np.pad(lens.profile, len(lens.profile), "constant")
        sim_profile = np.expand_dims(
            sim_profile, axis=0
        )  # TODO: remove this once 2D profiles are implemented...
        # fig = plt.figure()
        # plt.title("SIM PROFILE")
        # plt.plot(sim_profile[0])
        # plt.show()
    else:
        pad_px = 0
        sim_profile = pad_simulation(lens, pad_px=(pad_px, pad_px))  # 2D only

        # utils.plot_lens_profile_2D(sim_profile)
        # plt.show()

    if options.verbose:
        print("-" * 20)
        print(f"Propagating Wavefront with Parameters")
        print(f"Lens: {lens}")
        print(f"Medium: {output_medium}")
        print(
            f"Slices: {n_slices}, Start: {start_distance:.2e}m, Finish: {finish_distance:.2e}m"
        )
        print(f"Passed Wavefront: {passed_wavefront is not None}")
        print(f"-" * 20)

    if lens.profile.ndim == 1:
        freq_arr = generate_squared_frequency_array(
            n_pixels=sim_profile.shape[-1], pixel_size=parameters.pixel_size
        )  # TODO: confirm correct for 2D
        plt.plot(freq_arr)
        plt.show()
    else:
        freq_arr = gen_sq_freq_arr_2d(
            n_pixels=sim_profile.shape[-1], pixel_size=parameters.pixel_size
        )

    delta = (
        lens.medium.refractive_index - output_medium.refractive_index
    ) * sim_profile
    phase = 2 * np.pi * delta / parameters.sim_wavelength # % (2 * np.pi)

    if lens.profile.ndim == 2:
        fig = plt.figure()
        plt.imshow(freq_arr, cmap="jet")
        plt.title("Freq Arr")
        plt.colorbar()
        utils.save_figure(
            fig, os.path.join(log_dir, str(stage_id), "freq.png")
        )
        plt.close(fig)

        fig = plt.figure()
        plt.imshow(delta)
        plt.title("Delta")
        plt.colorbar()
        utils.save_figure(
            fig, os.path.join(log_dir, str(stage_id), "delta.png")
        )
        plt.close(fig)
        fig = plt.figure()
        plt.imshow(phase)
        plt.title("Phase")
        plt.colorbar()
        utils.save_figure(
            fig, os.path.join(log_dir, str(stage_id), "phase.png")
        )
        plt.close(fig)

    # only amplifiy the first stage propagation
    A = parameters.A if passed_wavefront is None else 1.0
    if passed_wavefront is not None:
        assert A == 1, "Amplitude is wrong"
        wavefront = A * np.exp(1j * phase) * passed_wavefront
    else:
        wavefront = A * np.exp(1j * phase)

    # padded area should be 0+0j
    if passed_wavefront is not None:  # TODO: convert to apeture mask
        wavefront[phase == 0] = 0 + 0j
    else:
        if pad_px !=0: # TODO: fix this properly
            wavefront[:, :pad_px] = 0 + 0j
            wavefront[:, -pad_px:] = 0 + 0j  # TODO: confirm this is correct for 2d?

    # QUERY: ^ dont think this is working properly for 2D

    # fourier transform of wavefront
    fft_wavefront = fftpack.fft2(wavefront)  # TODO: change to np

    # sim = np.ones(shape=(n_slices, *sim_profile.shape))
    top_down_view = np.zeros(shape=(n_slices, sim_profile.shape[1]))
    side_on_view = np.zeros(shape=(n_slices, sim_profile.shape[0]))

    if DEBUG:
        print(f"sim_profile.shape={sim_profile.shape}")
        print(f"freq_arr.shape={freq_arr.shape}")
        print(f"delta.shape={delta.shape}")
        print(f"phase.shape={phase.shape}")
        print(f"wavefront.shape={wavefront.shape}")
        print(f"fft_wavefront.shape={fft_wavefront.shape}")
        # print(f"sim.shape={sim.shape}")s
        print(f"top_down_view.shape={top_down_view.shape}")
        print(f"side_on_view.shape={side_on_view.shape}")

        # check the freq arr was created correctly
        assert freq_arr.shape[-1] == wavefront.shape[-1]
        if passed_wavefront is not None:
            assert not np.array_equal(np.unique(wavefront), [0 + 0j])  # non empty sim

    distances = np.linspace(start_distance, finish_distance, n_slices)
    prop_progress_bar = tqdm(distances, leave=False)
    for i, z in enumerate(prop_progress_bar):
        prop_progress_bar.set_description(
            f"Propagating Wavefront at Distance {z:.4f} / {distances[-1]:.4f}m"
        )
        prop = np.exp(1j * output_medium.wave_number * z) * np.exp(
            (-1j * 2 * np.pi ** 2 * z * freq_arr) / output_medium.wave_number
        )
        # prop = np.exp(
        #     (-1j * 2 * np.pi ** 2 * z * freq_arr) / output_medium.wave_number
        # )
        propagation = fftpack.ifft2(prop * fft_wavefront)

        output = np.sqrt(propagation.real ** 2 + propagation.imag ** 2) ** 2

        # sim[i] = np.round(output, 8)
        rounded_output = output #np.round(output, 5)

        # save output
        # TODO: use npz (compressed)
        save_path = os.path.join(
            log_dir, str(stage_id), f"{z*1000:.8f}mm.npy"
        )
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        np.save(save_path, rounded_output)

        if lens.profile.ndim==2:
            # calculate views
            centre_px_h = output.shape[0] // 2
            centre_px_v = output.shape[1] // 2
            top_down_slice = rounded_output[centre_px_v, :]
            side_on_slice = rounded_output[:, centre_px_h]

            # append views
            top_down_view[i, :] = top_down_slice
            side_on_view[i, :] = side_on_slice
        else:
            top_down_view[i, :] = rounded_output




    ################## SAVE ##################

    # save top-down
    # fig = plt.figure()
    # plt.imshow(
    #     top_down_view, interpolation="spline36", aspect="auto", cmap="jet",
    # )
    # plt.title("Top Down View")
    # plt.colorbar()
    fig = utils.plot_simulation(
        top_down_view,
        top_down_view.shape[1],
        top_down_view.shape[0],
        parameters.pixel_size,
        start_distance,
        finish_distance,
    )

    utils.save_figure(
        fig, os.path.join(log_dir, str(stage_id), "topdown.png")
    )
    plt.close(fig)

    fig = utils.plot_simulation(
        side_on_view,
        side_on_view.shape[1],
        side_on_view.shape[0],
        parameters.pixel_size,
        start_distance,
        finish_distance,
    )
    utils.save_figure(
        fig, os.path.join(log_dir, str(stage_id), "sideon.png")
    )
    plt.close(fig)

    return propagation

In [15]:


log_dir = "test"


parameters = SimulationParameters(
    A=100, pixel_size=0.1e-6, sim_width=500e-6, sim_wavelength=488.e-9
)

options = SimulationOptions(
    save=False, save_plot=False, verbose=False, debug=False
)

# stage 0
lens0 = Lens(diameter=parameters.sim_width, height=0, exponent=0, medium=Medium(2.348))
sim_stage = SimulationStage(
    lens=lens0,
    output=Medium(2.348),
    n_slices=2, 
    start_distance=0, 
    finish_distance=0.1e-3,
    options=None
)
passed_wavefront = None
lens0.generate_profile(parameters.pixel_size)
stage_id = "0"


clean_prop = propagate_wavefront(sim_stage, None)

In [11]:
# stage 1
lens1 = Lens(diameter=parameters.sim_width, height=75e-6, exponent=2.0, medium=Medium(2.348))
output=Medium(1.5)
sim_stage = SimulationStage(
    lens=lens1,
    output=output,
    n_slices=20, 
    start_distance=0, 
    finish_distance=2*calculate_equivalent_focal_distance(lens1, output),
    options=None
)
lens1.generate_profile(parameters.pixel_size)
passed_wavefront = clean_prop
stage_id = 1

propagate_wavefront(sim_stage, clean_prop)


LENS NDIM:  1


array([[-74.02488926-103.5332593j , -51.73539832 -71.40015277j,
        -51.51652999 -37.83539769j, ..., -51.60451484 -37.7798113j ,
        -51.68584057 -71.22268421j, -74.12978972-103.56484536j],
       [-51.73539832 -71.40015277j, -36.36105575 -49.20304285j,
        -35.83124943 -25.86241113j, ..., -35.59960979 -25.79454008j,
        -36.03864179 -49.48646325j, -51.67411543 -71.15215752j],
       [-51.51652999 -37.83539769j, -35.83124943 -25.86241113j,
        -30.21289391  -9.98824054j, ..., -30.34585767  -9.81062414j,
        -35.70016227 -25.80471937j, -51.57849852 -37.79554675j],
       ...,
       [-51.60451484 -37.7798113j , -35.59960979 -25.79454008j,
        -30.34585767  -9.81062414j, ..., -30.29829189  -9.98808279j,
        -35.63453798 -25.84582809j, -51.54207155 -37.79905965j],
       [-51.68584057 -71.22268421j, -36.03864179 -49.48646325j,
        -35.70016227 -25.80471937j, ..., -35.63453798 -25.84582809j,
        -36.00453397 -49.36721924j, -51.7306149  -71.25281748j]